In [ ]:
import pandas as pd

In [ ]:
from model_utils.util_global_struct import process_bb_old_to_new
import model_utils.utils_s2 as us2 # TODO merge s2 util
from model_utils.utils_nn_s2 import predict_wrapper, stem2db_str

In [ ]:
predictor_s2 = us2.Predictor('v0.2')

In [ ]:
# df = pd.read_pickle('data/rfam_t0p01_k1.pkl.gz')
df = pd.read_pickle('data/rfam_t0p1_k1.pkl.gz')

In [ ]:
len(df)

In [ ]:
len(df[df['len'] <= 100])

In [ ]:
# df.sort_values(by='len')[140:]

In [ ]:
# row = df.iloc[134]
row = df[df['seq_id'] == 'RF00488_B'].iloc[0]

In [ ]:
row

In [ ]:
df_target = process_bb_old_to_new(row['bounding_boxes'])
df_target

In [ ]:
df_pred_stem = pd.DataFrame(row['bb_stem'])
len(df_pred_stem)

In [ ]:
df_pred_iloop = pd.DataFrame(row['bb_iloop'])
len(df_pred_iloop)

In [ ]:
df_pred_hloop = pd.DataFrame(row['bb_hloop'])
len(df_pred_hloop)

In [ ]:
df_target.columns

In [ ]:
pd.merge(df_target[df_target['bb_type'] == 'stem'], df_pred_stem, 
         on=['bb_x', 'bb_y', 'siz_x', 'siz_y'], how='left')

In [ ]:
pd.merge(df_target[df_target['bb_type'] == 'iloop'], df_pred_iloop, 
         on=['bb_x', 'bb_y', 'siz_x', 'siz_y'], how='left')

In [ ]:
pd.merge(df_target[df_target['bb_type'] == 'hloop'], df_pred_hloop, 
         on=['bb_x', 'bb_y', 'siz_x', 'siz_y'], how='left')

In [ ]:
df_pred

In [ ]:
print("s1 bounding boxes (before pruning): {} stems, {} iloops, {} hloops".format(len(df_pred_stem), len(df_pred_iloop), len(df_pred_hloop)))

In [ ]:
%%time
# s2 prediction
df_pred = predict_wrapper(df_pred_stem, df_pred_iloop, df_pred_hloop, 
                          discard_ns_stem=True, min_hloop_size=2, 
                          seq=row['seq'], m_factor=1, predictor=predictor_s2)

In [ ]:
# for forna display
db_str_target = stem2db_str(df_stem=df_target[df_target['bb_type'] == 'stem'], seq_len=row['len'])
db_str_pred = stem2db_str(df_stem=df_pred[df_pred['bb_type'] == 'stem'], seq_len=row['len'])
print(row['seq_id'])
print(">s1\n{}\n{}\n\n>s2\n{}\n{}".format(row['seq'], db_str_target, row['seq'], db_str_pred))

In [ ]:
# RL?